# Runs for 2023 Internal Lysis paper
## Re-run for more binding data

In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY310,
)

In [2]:
import os
import subprocess
import sys

sys.path.insert(1, os.path.abspath(".."))

import GooseSLURM as gs
import numpy as np

import lysis

In [3]:
scenario_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("total_molecules", int),
        ("pore_size", float),  # in microns. Code requires cm
        ("fiber_diameter", float),  # in microns. Needs to be added to pore_size to get grid_node_distance
        ("cols", int),
        ("rows", int),
        ("empty_rows", int),
        ("forced_unbind", float),
    ]
)
mechanism_type = np.dtype([("descriptor", np.str_, 40), ("executable", np.str_, 40)])
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("scenario", np.str_, 40),
        ("mechanism", np.str_, 40),
        ("seed", int),
        ("running_time", int),
        ("save_interval", int),
    ]
)
diameter_code = {
    72.7: "Q2",
    145.4: "Q4",
}

In [4]:
# seed sequence entropy: 3881821051554698152964433817123076384
scenarios = np.array(
    [
        ("TN-L_9350", 9_350, 1.0135, 72.7, 93, 93, 0, 0.0852),
        ("TN-L_307", 307, 1.0135, 72.7, 93, 93, 0, 0.0852),
        ("TN-D_684", 684, 0.22, 72.7, 342, 116, 0, 0.0852),
        ("TN-D_9350", 9_350, 0.22, 72.7, 342, 116, 0, 0.0852),
        ("TN-D_307", 307, 0.22, 72.7, 342, 116, 0, 0.0852),
        ("TK-L_3042", 3_042, 1.0135, 145.4, 87, 29, 0, 0.0729129),
        ("TK-L_9350", 9_350, 1.0135, 145.4, 87, 29, 0, 0.0729129),
        ("TK-L_307", 307, 1.0135, 145.4, 87, 29, 0, 0.0729129),
        ("TK-D_307", 307, 0.22, 145.4, 274, 42, 0, 0.0729129),
        ("TK-D_9350", 9350, 0.22, 145.4, 274, 42, 0, 0.0729129),
    ],
    dtype=scenario_type,
)
mechanisms = np.array(
    [
        ("Into and along - Internal", "macro_diffuse_into_and_along__internal"),
    ],
    dtype=mechanism_type,
)

In [5]:
in_file_code = "_PLG2_tPA01_{input_code}.dat"
out_file_code = "_{scenario_code}"

In [6]:
group_code = "2023-12-04-10"
runs = np.array(
    [
        (group_code + "00", "TN-L_9350", "Into and along - Internal", 1_034_836_197, 0, 10),
        (group_code + "01", "TN-L_307", "Into and along - Internal", 2_464_079_704, 0, 100),
        (group_code + "02", "TN-D_684", "Into and along - Internal", 982_919_027, 0, 100),
        (group_code + "03", "TN-D_9350", "Into and along - Internal", 129_314_213, 0, 10),
        (group_code + "04", "TN-D_307", "Into and along - Internal", 2_705_873_059, 0, 100),
        (group_code + "05", "TK-L_3042", "Into and along - Internal", 2_104_158_012, 0, 10),
        (group_code + "06", "TK-L_9350", "Into and along - Internal", 2_608_028_918, 0, 10),
        (group_code + "07", "TK-L_307", "Into and along - Internal", 1_819_293_232, 0, 100),
        (group_code + "08", "TK-D_307", "Into and along - Internal", 3_128_495_922, 0, 100),
        (group_code + "09", "TK-D_9350", "Into and along - Internal", 106_672_317, 0, 10),
    ],
    dtype=run_type,
)

In [7]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    p = {
        "total_molecules": int(scen["total_molecules"]),
        "pore_size": scen["pore_size"] / 10_000,  # convert to cm
        "grid_node_distance": scen["fiber_diameter"] / 1_000 + scen["pore_size"],
        "cols": int(scen["cols"]),
        "rows": int(scen["rows"]),
        "empty_rows": int(scen["empty_rows"]),
        "forced_unbind": scen["forced_unbind"],
        "seed": int(run["seed"]),
        "total_time": int(run["running_time"]),
        "save_interval": int(run["save_interval"]),
        "macro_version": "macro_diffuse_into_and_along__internal",
    }
    e.initialize_macro_param(p)
    e.to_file()

    with open(os.path.join(e.os_path, "README.md"), "w") as file:
        file.write(
            f""" ## {e.experiment_code}
Internal lysis experiments
Array version

Uses Fortran Macroscale code with
    -  Diffuse into and along
    -  'restricted move' bug correction
    -  'passerby molecule' bug correction
    -  addition of extra output data
    -  changing the termination criteria from 'fixed time' to 'all fibers degraded' AND '95% of molecules have passed through'
    -  changing macro-unbind wait time to remaining leaving time.
    -  changing the molecule start location calculation
    -  eliminating unused data variables
    -  Moved degraded fiber check into molecule loop
    -  Removed "degrade" array and use "t_degrade" instead
    -  Read in "neighborc" array generated in Python
    -  Output molecule bind/unbind times for transit time calculations
    -  Output molecule bind/unbind locations.

This set of data is from the "{run['scenario']}" set.
Should have identical results to 2023-06-09-10{e.experiment_code[-2:]}
"""
        )

    result = subprocess.run(
        ["cp", os.path.join("src", "fortran", mech["executable"] + ".f90"), e.os_path],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__internal.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1000'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__internal.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1001'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__internal.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1002'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__internal.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1003'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__internal.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1004'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_di

In [8]:
folder_vars = """
homedir="/home/bpaynter/git/UCO-OpResearch/lysis"
workdir="/tmp/bpaynter/${SLURM_ARRAY_JOB_ID}-${SLURM_ARRAY_TASK_ID}"
datadir="${workdir}/data/${exp_code}"
"""

transfer_script = """
# 1. Transfer to node 
# ====================

# create/empty the temporary directory on the compute node
if [ ! -d "${workdir}" ]; then
  mkdir -p "${workdir}"
else
  echo rm -ri "${workdir}"/*
fi

mkdir -p "${datadir}"

cp ${homedir}/data/${exp_code}/*${in_code} ${datadir}/
cp ${homedir}/data/${exp_code}/params.json ${datadir}/
cp ${homedir}/bin/${fort_executable} ${workdir}/
"""

return_script = """
# 2. Function to transfer back to the head node 
# ==============================================

# define clean-up function
function clean_up {
  sim=$(printf "%02d" ${SLURM_ARRAY_TASK_ID})
  # - copy everything from the temporary directory on the compute-node
  mkdir -p ${homedir}/data/${exp_code}/$sim
  cp -prf "${datadir}"/*${out_code}* ${homedir}/data/${exp_code}/$sim/
  cp -prf "${datadir}"/*.txt ${homedir}/data/${exp_code}/$sim/
  # - erase the temporary directory from the compute-node
  rm -rf "${workdir}"/*
  rm -rf "${workdir}"
  # - exit the script
  exit
}

# call "clean_up" function when this script exits, it is run even if SLURM cancels the job
trap 'clean_up' EXIT
"""
execute_script = """
# 3. Execute 
# ===========

source /home/bpaynter/.bashrc
source /home/bpaynter/lysis.sh
cd ${homedir}/src/python

python -u fortran_run.py \
    --in_code ${in_code} \
    --out_code ${out_code}.dat \
    -n $SLURM_ARRAY_TASK_ID \
    --cwd ${workdir} \
    ${workdir}/${fort_executable} \
    ${exp_code}
    
sleep 10
"""

In [9]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    e.read_file()
    run_vars = f"""
exp_code="{e.experiment_code}"
in_code="{in_file_code.format(input_code=diameter_code[scen['fiber_diameter']])}"
out_code="{out_file_code.format(scenario_code=run['scenario'])}"
fort_executable="{mech['executable']}"
"""
    # job-options
    sbatch = {
        "job-name": f"lysis-{e.experiment_code}",
        "out": os.path.join(e.os_path, "job.slurm-%A-%a.out"),
        "array": f"0-{e.macro_params.total_trials-1}",
        "nodes": 1,
        "mem": 3096,
        "ntasks": 1,
        "cpus-per-task": 1,
        "exclusive=user": "",
        "partition": "long",
    }
    script = [run_vars, folder_vars, transfer_script, return_script, execute_script]
    with open(os.path.join(e.os_path, "job.slurm"), "w") as file:
        file.write(gs.scripts.plain(script, **sbatch))
    result = subprocess.run(
        ["sbatch", os.path.join(e.os_path, "job.slurm")],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1000/job.slurm'], returncode=0, stdout=b'Submitted batch job 2220030\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1001/job.slurm'], returncode=0, stdout=b'Submitted batch job 2220031\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1002/job.slurm'], returncode=0, stdout=b'Submitted batch job 2220032\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1003/job.slurm'], returncode=0, stdout=b'Submitted batch job 2220033\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1004/job.slurm'], returncode=0, stdout=b'Submitted batch job 2220034\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-12-04-1005/job.slurm'], returncode=0, std